In [331]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import classification_report

import time

In [332]:
np.random.seed(42)
pd.set_option('display.max_rows', None)

In [333]:
df = pd.read_csv("Data/train.csv")
df_test = pd.read_csv("Data/test.csv")

In [334]:
df['Embarked'].fillna('S', inplace=True)
df_test['Fare'].fillna(26.55, inplace=True)
for i,r in df.iterrows():
    if pd.isnull(r['Age']):
        print(df[df['Pclass'] == r['Pclass']]['Age'].median() ,r['Pclass'])
        df.loc[df.index[i],'Age'] = df[df['Pclass'] == r['Pclass']]['Age'].median()

for i,r in df_test.iterrows():
    if pd.isnull(r['Age']):
        print(df_test[df_test['Pclass'] == r['Pclass']]['Age'].median() ,r['Pclass'])
        df_test.loc[df_test.index[i],'Age'] = df_test[df_test['Pclass'] == r['Pclass']]['Age'].median()

24.0 3
29.0 2
24.0 3
24.0 3
24.0 3
24.0 3
37.0 1
24.0 3
24.0 3
24.0 3
24.0 3
24.0 3
24.0 3
24.0 3
37.0 1
37.0 1
24.0 3
24.0 3
24.0 3
24.0 3
24.0 3
24.0 3
24.0 3
24.0 3
24.0 3
24.0 3
24.0 3
24.0 3
24.0 3
24.0 3
24.0 3
24.0 3
37.0 1
37.0 1
24.0 3
24.0 3
29.0 2
37.0 1
24.0 3
24.0 3
24.0 3
24.0 3
24.0 3
24.0 3
24.0 3
24.0 3
24.0 3
24.0 3
24.0 3
37.0 1
24.0 3
24.0 3
37.0 1
24.0 3
29.0 2
37.0 1
37.0 1
37.0 1
24.0 3
24.0 3
29.0 2
24.0 3
37.0 1
24.0 3
24.0 3
37.0 1
24.0 3
24.0 3
37.0 1
24.0 3
24.0 3
24.0 3
24.0 3
24.0 3
24.0 3
37.0 1
24.0 3
24.0 3
24.0 3
24.0 3
24.0 3
29.0 2
24.0 3
24.0 3
24.0 3
24.0 3
24.0 3
24.0 3
24.0 3
24.0 3
37.0 1
24.0 3
24.0 3
29.0 2
24.0 3
24.0 3
37.0 1
29.0 2
24.0 3
24.0 3
24.0 3
24.0 3
24.0 3
37.0 1
24.0 3
24.0 3
24.0 3
24.0 3
37.0 1
24.0 3
24.0 3
24.0 3
29.0 2
24.0 3
37.0 1
24.0 3
24.0 3
24.0 3
24.0 3
24.0 3
24.0 3
24.0 3
24.0 3
24.0 3
29.0 2
24.0 3
24.0 3
37.0 1
24.0 3
24.0 3
24.0 3
24.0 3
37.0 1
24.0 3
24.0 3
24.0 3
24.0 3
24.0 3
24.0 3
24.0 3
37.0 1
29.0 2
24.0 3

In [335]:
print(df.isnull().sum(),df_test.isnull().sum())

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
dtype: int64 PassengerId      0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          327
Embarked         0
dtype: int64


In [261]:
merge = pd.concat([df,df_test],ignore_index=1)

print(merge['Cabin Letter'].value_counts())

Cabin Letter
N    1014
C      94
B      65
D      46
E      41
A      22
F      21
G       5
T       1
Name: count, dtype: int64


In [260]:
df['Cabin Letter']=df['Cabin'].apply(lambda x: x[0] if not pd.isna(x) else 'N')
df_test['Cabin Letter']=df_test['Cabin'].apply(lambda x: x[0] if not pd.isna(x) else 'N')

In [262]:
df['Cabin N']=df['Cabin Letter'].apply(lambda x: 1 if x == 'N' else 0)
df['Cabin C']=df['Cabin Letter'].apply(lambda x: 1 if x == 'C' else 0)
df['Cabin B']=df['Cabin Letter'].apply(lambda x: 1 if x == 'B' else 0)
df['Cabin D']=df['Cabin Letter'].apply(lambda x: 1 if x == 'D' else 0)
df['Cabin E']=df['Cabin Letter'].apply(lambda x: 1 if x == 'E' else 0)
df['Cabin A']=df['Cabin Letter'].apply(lambda x: 1 if x == 'A' else 0)
df['Cabin F']=df['Cabin Letter'].apply(lambda x: 1 if x == 'F' else 0)
df['Cabin G']=df['Cabin Letter'].apply(lambda x: 1 if x == 'G' else 0)
df['Cabin T']=df['Cabin Letter'].apply(lambda x: 1 if x == 'T' else 0)

df_test['Cabin N']=df_test['Cabin Letter'].apply(lambda x: 1 if x == 'N' else 0)
df_test['Cabin C']=df_test['Cabin Letter'].apply(lambda x: 1 if x == 'C' else 0)
df_test['Cabin B']=df_test['Cabin Letter'].apply(lambda x: 1 if x == 'B' else 0)
df_test['Cabin D']=df_test['Cabin Letter'].apply(lambda x: 1 if x == 'D' else 0)
df_test['Cabin E']=df_test['Cabin Letter'].apply(lambda x: 1 if x == 'E' else 0)
df_test['Cabin A']=df_test['Cabin Letter'].apply(lambda x: 1 if x == 'A' else 0)
df_test['Cabin F']=df_test['Cabin Letter'].apply(lambda x: 1 if x == 'F' else 0)
df_test['Cabin G']=df_test['Cabin Letter'].apply(lambda x: 1 if x == 'G' else 0)
df_test['Cabin T']=df_test['Cabin Letter'].apply(lambda x: 1 if x == 'T' else 0)

In [336]:
df.drop(columns='PassengerId', inplace=True)
df_test.drop(columns='PassengerId', inplace=True)

df.drop(columns='Cabin', inplace=True)
df_test.drop(columns='Cabin', inplace=True)

"""df.drop(columns='Cabin Letter', inplace=True)
df_test.drop(columns='Cabin Letter', inplace=True)"""

"df.drop(columns='Cabin Letter', inplace=True)\ndf_test.drop(columns='Cabin Letter', inplace=True)"

In [337]:
df['Family']=df['Parch']+df['SibSp']+1
df_test['Family']=df_test['Parch']+df_test['SibSp']+1

In [338]:
df.drop(columns='Parch', inplace=True)
df_test.drop(columns='Parch', inplace=True)

df.drop(columns='SibSp', inplace=True)
df_test.drop(columns='SibSp', inplace=True)

In [339]:
df['Family Solo'] = df['Family'].apply(lambda x: 1 if x == 1 else 0)
df['Family Small'] = df['Family'].apply(lambda x: 1 if x >1 and x <4 else 0)
df['Family Normal'] = df['Family'].apply(lambda x: 1 if x >= 4 and x <6 else 0)
df['Family Big'] = df['Family'].apply(lambda x: 1 if x >=6 else 0)

df_test['Family Solo'] = df_test['Family'].apply(lambda x: 1 if x == 1 else 0)
df_test['Family Small'] = df_test['Family'].apply(lambda x: 1 if x >1 and x <4 else 0)
df_test['Family Normal'] = df_test['Family'].apply(lambda x: 1 if x >= 4 and x <6 else 0)
df_test['Family Big'] = df_test['Family'].apply(lambda x: 1 if x >=6 else 0)

In [340]:
df['Embarked S'] = df['Embarked'].apply(lambda x: 1 if x == 'S' or type(x) != str else 0)
df['Embarked C'] = df['Embarked'].apply(lambda x: 1 if x == 'C' else 0)
df['Embarked Q'] = df['Embarked'].apply(lambda x: 1 if x == 'Q' else 0)

df_test['Embarked S'] = df_test['Embarked'].apply(lambda x: 1 if x == 'S' or type(x) != str else 0)
df_test['Embarked C'] = df_test['Embarked'].apply(lambda x: 1 if x == 'C' else 0)
df_test['Embarked Q'] = df_test['Embarked'].apply(lambda x: 1 if x == 'Q' else 0)


In [341]:
df['Fare Per P'] = df['Fare']/df['Family']
df_test['Fare Per P'] = df_test['Fare']/df_test['Family']

In [342]:
df['Fare Cheap']= df['Fare Per P'].apply(lambda x: 1 if x < 20 else 0)
df['Fare Mid']= df['Fare Per P'].apply(lambda x: 1 if x >= 20 and x < 70 else 0)
df['Fare Expensive']= df['Fare Per P'].apply(lambda x: 1 if x >= 70 and x < 200 else 0)
df['Fare Ultra']= df['Fare Per P'].apply(lambda x: 1 if x >= 200 else 0)

df_test['Fare Cheap']= df_test['Fare Per P'].apply(lambda x: 1 if x < 20 else 0)
df_test['Fare Mid']= df_test['Fare Per P'].apply(lambda x: 1 if x >= 20 and x < 70 else 0)
df_test['Fare Expensive']= df_test['Fare Per P'].apply(lambda x: 1 if x >= 70 and x < 200 else 0)
df_test['Fare Ultra']= df_test['Fare Per P'].apply(lambda x: 1 if x >= 200 else 0)

In [343]:
df['Title'] = df['Name'].apply(lambda x: x.split('.')[0].split()[-1])
df_test['Title'] = df_test['Name'].apply(lambda x: x.split('.')[0].split()[-1])

In [344]:
df['Passanger'] = df['Title'].apply(lambda x: 1 if x in ['Mrs', 'Mr', 'Miss', 'Ms', 'Mme', 'Mlle', 'Master'] else 0)
df['Royale'] = df['Title'].apply(lambda x: 1 if x in ['Lady', 'Sir', 'Don', 'Jonkheer', 'Countess', 'Dona'] else 0)
df['Crew'] = df['Title'].apply(lambda x: 1 if x in ['Major', 'Col', 'Rev', 'Capt', 'Dr'] else 0)

df_test['Passanger'] = df_test['Title'].apply(lambda x: 1 if x in ['Mrs', 'Mr', 'Miss', 'Ms', 'Mme', 'Mlle', 'Master'] else 0)
df_test['Royale'] = df_test['Title'].apply(lambda x: 1 if x in ['Lady', 'Sir', 'Don', 'Jonkheer', 'Countess', 'Dona'] else 0)
df_test['Crew'] = df_test['Title'].apply(lambda x: 1 if x in ['Major', 'Col', 'Rev', 'Capt', 'Dr'] else 0)

In [345]:
df['Child']= df['Age'].apply(lambda x: 1 if x < 16 else 0)
df['Young']= df['Age'].apply(lambda x: 1 if x >= 16 and x < 28 else 0)
df['Mature']= df['Age'].apply(lambda x: 1 if x >= 28 and x < 55 else 0)
df['Old']= df['Age'].apply(lambda x: 1 if x >= 55 else 0)

df_test['Child']= df_test['Age'].apply(lambda x: 1 if x < 16 else 0)
df_test['Young']= df_test['Age'].apply(lambda x: 1 if x >= 16 and x < 28 else 0)
df_test['Mature']= df_test['Age'].apply(lambda x: 1 if x >= 28 and x < 55 else 0)
df_test['Old']= df_test['Age'].apply(lambda x: 1 if x >= 55 else 0)

In [346]:
df.drop(columns='Name', inplace=True)
df_test.drop(columns='Name', inplace=True)

df.drop(columns='Age', inplace=True)
df_test.drop(columns='Age', inplace=True)

df.drop(columns='Ticket', inplace=True)
df_test.drop(columns='Ticket', inplace=True)

df.drop(columns='Title', inplace=True)
df_test.drop(columns='Title', inplace=True)

df.drop(columns='Fare', inplace=True)
df_test.drop(columns='Fare', inplace=True)

df.drop(columns='Fare Per P', inplace=True)
df_test.drop(columns='Fare Per P', inplace=True)

df.drop(columns='Embarked', inplace=True)
df_test.drop(columns='Embarked', inplace=True)


In [347]:
df.drop(columns='Family', inplace=True)
df_test.drop(columns='Family', inplace=True)

In [348]:
df['Class 1'] = df['Pclass'].apply(lambda x: 1 if x == 1 else 0)
df['Class 2'] = df['Pclass'].apply(lambda x: 1 if x == 2 else 0)
df['Class 3'] = df['Pclass'].apply(lambda x: 1 if x == 3 else 0)

df_test['Class 1'] = df_test['Pclass'].apply(lambda x: 1 if x == 1 else 0)
df_test['Class 2'] = df_test['Pclass'].apply(lambda x: 1 if x == 2 else 0)
df_test['Class 3'] = df_test['Pclass'].apply(lambda x: 1 if x == 3 else 0)

In [349]:
df['Male'] = df['Sex'].apply(lambda x: 1 if x == 'male' else 0)
df['Female'] = df['Sex'].apply(lambda x: 1 if x == 'female' else 0)

df_test['Male'] = df_test['Sex'].apply(lambda x: 1 if x == 'male' else 0)
df_test['Female'] = df_test['Sex'].apply(lambda x: 1 if x == 'female' else 0)


In [350]:
df.drop(columns='Sex', inplace=True)
df_test.drop(columns='Sex', inplace=True)

df.drop(columns='Pclass', inplace=True)
df_test.drop(columns='Pclass', inplace=True)

In [293]:
df.drop(columns='Cabin T', inplace=True)
df_test.drop(columns='Cabin T', inplace=True)

In [295]:
df.drop(columns='Embarked S', inplace=True)
df_test.drop(columns='Embarked S', inplace=True)

df.drop(columns='Embarked Q', inplace=True)
df_test.drop(columns='Embarked Q', inplace=True)

df.drop(columns='Embarked C', inplace=True)
df_test.drop(columns='Embarked C', inplace=True)

In [308]:
df.drop(columns='Cabin B', inplace=True)
df_test.drop(columns='Cabin B', inplace=True)
df.drop(columns='Cabin D', inplace=True)
df_test.drop(columns='Cabin D', inplace=True)
df.drop(columns='Cabin C', inplace=True)
df_test.drop(columns='Cabin C', inplace=True)
df.drop(columns='Cabin E', inplace=True)
df_test.drop(columns='Cabin E', inplace=True)
df.drop(columns='Cabin F', inplace=True)
df_test.drop(columns='Cabin F', inplace=True)
df.drop(columns='Cabin G', inplace=True)
df_test.drop(columns='Cabin G', inplace=True)

In [310]:
df.drop(columns='Cabin A', inplace=True)
df_test.drop(columns='Cabin A', inplace=True)

In [351]:
df.T

,0,1,2,3,4,5,6,7,8,9,...,881,882,883,884,885,886,887,888,889,890
Survived,0,1,1,1,0,0,0,0,1,1,...,0,0,0,0,0,0,1,0,1,0
Family Solo,0,0,1,0,1,1,1,0,0,0,...,1,1,1,1,0,1,1,0,1,1
Family Small,1,1,0,1,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
Family Normal,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
Family Big,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
Embarked S,1,0,1,1,1,0,1,1,1,0,...,1,1,1,1,0,1,1,1,0,0
Embarked C,0,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
Embarked Q,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
Fare Cheap,1,0,1,0,1,1,0,1,1,1,...,1,1,1,1,1,1,0,1,0,1
Fare Mid,0,1,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,1,0


In [352]:
X= df.drop('Survived', axis=1)
y= df.Survived.values

In [353]:
from sklearn.model_selection import KFold

kfold_splitter = KFold(n_splits=5, random_state = 42, shuffle=True)

In [356]:
modelRF= RandomForestClassifier(n_estimators=450, random_state=42,min_samples_split=6,min_samples_leaf=2,max_depth=None)
modelRF.fit(X_train, y_train)

v_acc = cross_val_score(modelRF,
                         X,
                         y,
                         cv=kfold_splitter, # 5-fold cross-validation
                         scoring="accuracy")

print(v_acc,np.mean(v_acc),modelRF.score(X_valid, y_valid))

[0.81564246 0.81460674 0.87640449 0.79775281 0.83146067] 0.8271734354403364 0.8156424581005587


In [354]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.2, random_state=42)

In [391]:
#finalModel=RandomForestClassifier(n_estimators=450, random_state=42,min_samples_split=6,min_samples_leaf=2,max_depth=None)
finalModel=svm.SVC()
#finalModel= KNeighborsClassifier(n_neighbors=5)
finalModel.fit(X,y)
pred = finalModel.predict(df_test)

In [392]:
write= pd.read_csv("Data/test.csv")
result=pd.DataFrame({"PassengerId":write['PassengerId'],"Survived":pred})

In [372]:
result

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
5,897,0
6,898,1
7,899,0
8,900,1
9,901,0


In [393]:
result.to_csv("res1.csv", index=False)

In [387]:
from sklearn import svm
clf = svm.SVC()
clf.fit(X_train, y_train)
clf.score(X_valid, y_valid)

0.8212290502793296

In [396]:
from sklearn.neighbors import KNeighborsClassifier

clf = KNeighborsClassifier(n_neighbors=13)
clf.fit(X_train, y_train)
clf.score(X_valid, y_valid)

0.8324022346368715